# 第二次作業

## Basline
請將提供的火災新聞描述中萃取出「起火點」、「起火原因」、「起火時間」、「是否有人死亡」，並以下方的 JSON 格式回傳，並轉成 Python Dict。

```json
{
    "起火點": "廚房",
    "起火原因": "電線起火",
    "起火時間": "早上9點",
    "死亡與否": false
}
```

提示：JSON Output Parser

In [1]:
!pip install accelerate
!pip install -U transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!pip install langchain
!pip install langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.0 MB/s eta 0:00:00


In [3]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00


In [76]:
import json
import re

In [5]:
import torch
from transformers import BitsAndBytesConfig

from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.schema import StrOutputParser

In [6]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_huggingface import ChatHuggingFace

In [7]:
from huggingface_hub import notebook_login

# chat model 需要使用 hf 的 token
notebook_login()

In [8]:
MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# 量化參數
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True)


In [9]:
# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15) )

# chat model 初始化
chat_llm = ChatHuggingFace(llm=llm)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
news = ["""
嘉義縣水上鄉三界村一處廢棄豬舍鐵皮屋昨深夜11點起火，地點位於台82線快速道路旁，濃煙竄天嚇壞許多民眾。警消獲報到場滅火，疑堆放大量塑膠製品迅速延燒，並傳出一名老人失聯，經徹夜搜索，消防人員今早9點多在旁民宅找到驚魂未定70歲男子，將他送醫。
據悉，水上鄉三界埔一處豬舍昨夜失火，現場疑堆放大量廢棄塑膠製品，導致火勢延燒迅速猛烈，鐵皮屋被燒到變形倒塌，火滅後仍持續悶燒，消防局出動20車、47人搶救，並出動大型機具開挖滅火，經徹夜9小時灌救滅火，初判燃燒面積約為2000平方公尺。
一名老翁平時住在廢棄豬舍附近，昨夜傳出疑似失聯，消防人員徹夜擴大搜索火場，並未在廢棄豬舍找到他，直到今天早上9點48分才在火場後方建築物2樓逃生梯發現，幸未被火勢延燒波及，只有建築物表面遭濃煙熏黑，老翁驚魂未定，但未受傷。
嘉義縣消防局表示，年約70歲男子待在後方建築物2樓逃生梯，意識清楚、無明顯外傷，給予救護處置後，由救護車預防性送往聖馬醫院診治。提醒民眾遇到火災切勿慌張，「小火快逃、濃煙關門」是火場逃生不二法門，且隨時注意逃生路徑，就能提高獲救機會。
""",
"""
台南市成功大學勝利校區體育館今天晚上9點多發生火災，消防局接獲報案派遣13車27人前往搶救，現場為2樓健身房的天花板管線冒煙，人員都已進行疏散，消防人員出水滅火，火勢約半小時後撲滅。
消防人員說，可能是體育館空調設備故障引起火災；台南市消防局說，起火真正原因需由消災調查科人員進一步鑑定查明。
台南市消防局救災救護指揮中心今晚9點32分接獲報案，成功大學勝利校區體育館發生火災，中心派遣東門等消防分隊13車27人前往搶救，消防人員到達現場回報為2樓健身房的天花板管線冒煙，已進行疏散。義消到達外觀有濃煙，校方有初期滅火。
據了解，2樓為健身房使用，無延燒之虞，館內人員皆撤出。消防人員出水降溫，及排煙。火勢於晚上10點3分撲滅。
""",
"""
超衰！50歲陳姓男子在高雄鳳山租屋，未料某天睡著時，床頭夾式檯燈電源線短路起火，火勢一發不可收拾，一路延燒到至少6處樓層，雖未釀成傷亡，但經消防火調科調查，證據都指向陳男釀災可能性最大，被5名租客聯合提告，法院審理後，將他依法判4個月有期徒刑，得易科罰金。
判決指出，50歲陳男高雄鳳山租屋，2023年初凌晨睡覺時，臥室床頭夾式檯燈電源線短路起火，瞬間燃燒周邊木質家具及電器，火勢一發不可收拾，一路延燒到至少6處樓層，造成建物客廳、廚房、廁所、床墊、家具等物被燒毀。
消防撲滅火勢後，雖未釀成人員傷亡，但受害5名房客聯合對陳男提告；經火調科調查，火災起火點位在陳男住處臥室靠客廳走廊床頭櫃附近，起火原因以電氣因素引燃可能性最大，他因此吃上官司。
陳男警詢及偵訊時，都坦承此事，被檢方依失火燒燬他人之物等罪嫌起訴，他也在審理過程中自白。
法官認為，陳男使用電器用品時，未注意夾式檯燈電源線使用狀況，導致火災發生，除燒燬自己租用住宅外，還波及隔鄰其他住戶，造成多人損失，考量坦承犯行，態度尚可，審理後，將他依犯失火燒燬現供人使用之住宅罪，處4個月有期徒刑，得易科罰金。
"""]

In [11]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


In [107]:
template = """火災新聞: {question}
        請將提供的火災新聞描述中萃取出
        起火點
        起火原因
        起火時間
        是否有人死亡(bool)
        並以JSON 格式回傳，是一個有4個中文key值的dictionary
        <答案>: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# 使用 LLM Chain 將 Prompt 與 LLM 串接起來
reason_llm_chain = LLMChain(prompt=prompt, llm=llm)
# reason_llm_chain = ChatHuggingFace(prompt=prompt, llm=llm)


### <法一>獨立function

In [108]:
def convert_to_json(input_str):
    _ = input_str.split('<答案>:')[1]
    # Removing newlines and leading/trailing whitespaces
    clean_str = _.replace('\n', '').strip().lower()

    # Converting the cleaned string to a JSON object
    json_obj = json.loads(clean_str)

    return json_obj

In [109]:
convert_to_json(reason_llm_chain.invoke({"question": news[0]})["text"])

{'起火點': '嘉義縣水上鄉三界村',
 '起火原因': '堆放大量塑膠製品',
 '起火時間': '昨日深夜11點',
 '是否有人死亡': False,
 '是否有人失蹤': True}

In [44]:
convert_to_json(reason_llm_chain.invoke({"question": news[1]})["text"])

{'起火點': '台南市成功大學勝利校區體育館',
 '起火原因': '可能是體育館空調設備故障引起火災',
 '起火時間': '今天晚上9點多',
 '是否有人死亡': False}

In [47]:
convert_to_json(reason_llm_chain.invoke({"question": news[2]})["text"])

{'起火點': '臥室头部摆放的夾式檯燈',
 '起火原因': '电器因电源线短路起火',
 '起火时间': '2023年初凌晨',
 '是否有人死亡': False,
 '法律处置': '被判4个月有期徒刑，得易科罰金'}

## <法二> 寫一個class加入llm_chain

In [120]:
class JsonProcessor:
    def __call__(self, inputs):
        # inputs 是一個 AIMessage 物件，獲取其內容
        response = inputs.content
        # 找到 <答案>: 之後的內容
        answer_start = response.find("<答案>:") + len("<答案>:")
        text = response[answer_start:].replace('\n', '').strip()
        try:
            # 使用正則表達式提取信息
            fire_location = re.search(r"起火點[:：]([\w\s]+)", text) # [\w\s]+ 是一個字類符號
            fire_cause = re.search(r"起火原因[:：]([\w\s]+)", text)
            fire_time = re.search(r"起火時間[:：]([\w\s]+)", text)
            death_info = re.search(r"是否有人死亡[:：](\w+)", text)

            # 構建结果字典
            response_json = {
                "起火點": fire_location.group(1).strip() if fire_location else "未知",
                "起火原因": fire_cause.group(1).strip() if fire_cause else "未知",
                "起火時間": fire_time.group(1).strip() if fire_time else "未知",
                "是否有人死亡": False if death_info and death_info.group(1).strip().lower() in ["false", "否"] else True
            }
        except Exception as e:
            # 如果轉換失敗，返回原始回應
            response_json = {"error": str(e), "response": response}
        return response_json

In [121]:
# 使用 LCEL 將 Prompt 與 LLM 等串接起來
llm_chain = prompt | chat_llm | JsonProcessor()

In [122]:
llm_chain.invoke({"question": news[0]})

{'起火點': '嘉義縣水上鄉三界村一處廢棄豬舍鐵皮屋起火原因',
 '起火原因': '疑堆放大量塑膠製品迅速延燒起火時間',
 '起火時間': '昨日深夜11點是否有人死亡',
 '是否有人死亡': False}

In [118]:
llm_chain.invoke({"question": news[1]})

{'起火點': '台南市成功大學勝利校區體育館起火原因',
 '起火原因': '可能是體育館空調設備故障起火時間',
 '起火時間': '今天晚上9點半左右是否有人死亡',
 '是否有人死亡': False}

In [119]:
llm_chain.invoke({"question": news[2]})

{'起火點': '陳男住處臥室靠客廳走廊床頭櫃附近起火原因',
 '起火原因': '電氣因素引燃可能性最大起火時間',
 '起火時間': '2023年初凌晨是否有人死亡',
 '是否有人死亡': False}

## Advance
延續練習一，是否有辦法先判斷問題是否與火災的資訊擷取有關

有關則去擷取「起火點」、「起火原因」、「起火時間」、「是否有人死亡」

若無關則回答此問題無法回答

`注意`：不要使用一個 Prompt 來解決問題，試著將透過 LCEL 串接兩個不同的 LLM Chain。

In [ ]:
news = [
"""
晶片大廠輝達（NVIDIA）執行長黃仁勳7日下午現身台北市內湖區基湖路上的輝達台灣分公司，據悉，此行他主要是為了與台灣區的駐派主管開會，而昨日才剛露面跟大家開心聚餐、喝到微醺的他，今日則重新以專業的態度及堅定的步伐，穩步快速的徑直進入公司，前去與其餘主管一起開會。
據悉，黃仁勳此行是為了參加「台北國際電腦展」才來台，而他6日晚間也特地在位於中山區的「欣葉台菜」與輝達內部的180名同仁，召開慶功宴，而黃仁勳自上個月底來台後，所到之處都也被粉絲包圍，媒體更是密切報導他的相關談話內容，而他昨晚現身街頭，也在台北市各地引起軒然大波。
據輝達台灣分公司官網顯示，團隊生產線從基礎發明到結合 GPU 的處理器，再到系統元件，乃至完全整合系統，同時將目標放在三大垂直市場：遊戲、專業視覺化與設計，以及高效能運算與巨量資料分析。針對每一垂直市場提供處理器、軟體、工具、行銷和專業知識的平台，另外更逐漸增加提供連線服務。
此外，團隊身為視覺運算的全球領導者，也銷售元件，並授權 IP 給希冀以豐富繪圖能力打造差異化裝置的各大 OEM，充分發揮自己這些市場所做的創新。而外傳黃仁勳即將於8日搭機離開台灣，也讓各界人士更加關注他的一舉一動。
""",
"""
嘉義縣水上鄉三界村一處廢棄豬舍鐵皮屋昨深夜11點起火，地點位於台82線快速道路旁，濃煙竄天嚇壞許多民眾。警消獲報到場滅火，疑堆放大量塑膠製品迅速延燒，並傳出一名老人失聯，經徹夜搜索，消防人員今早9點多在旁民宅找到驚魂未定70歲男子，將他送醫。
據悉，水上鄉三界埔一處豬舍昨夜失火，現場疑堆放大量廢棄塑膠製品，導致火勢延燒迅速猛烈，鐵皮屋被燒到變形倒塌，火滅後仍持續悶燒，消防局出動20車、47人搶救，並出動大型機具開挖滅火，經徹夜9小時灌救滅火，初判燃燒面積約為2000平方公尺。
一名老翁平時住在廢棄豬舍附近，昨夜傳出疑似失聯，消防人員徹夜擴大搜索火場，並未在廢棄豬舍找到他，直到今天早上9點48分才在火場後方建築物2樓逃生梯發現，幸未被火勢延燒波及，只有建築物表面遭濃煙熏黑，老翁驚魂未定，但未受傷。
嘉義縣消防局表示，年約70歲男子待在後方建築物2樓逃生梯，意識清楚、無明顯外傷，給予救護處置後，由救護車預防性送往聖馬醫院診治。提醒民眾遇到火災切勿慌張，「小火快逃、濃煙關門」是火場逃生不二法門，且隨時注意逃生路徑，就能提高獲救機會。
""",
"""
對貓咪來說，貓跳台就是一個可以休息、運動的空間，是許多貓奴家裡的必備物品。根據日本寵物網《ねこちゃんホンポ》專欄作家指出，貓跳台的擺放位置非常重要，如果放在下述４個NG地點，不僅可能傷害到貓咪，連人類也有受傷的風險，鏟屎官一定要留意。
1.冷氣直吹：很多飼主都會忽略冷氣的位置，把貓跳台放在冷氣出風口附近，冷氣、暖氣對著貓跳台直直吹，貓咪很容易感冒打噴嚏、流鼻涕，還可能引起腹瀉等健康問題。
2.靠近電暖器、電暖扇：無論貓咪的運動神經有多好，都還是會不小心沒踩穩從跳台上摔下來，如果墜落的地方有擺放電暖器、電暖扇等發熱型電器，貓咪可能會被燙傷，嚴重也可能引起火災。
3.靠近沒有防護的窗戶：很多飼主都會把貓跳台放在窗戶附近，讓貓咪曬太陽、看風景。但是如果貓跳台附近的窗戶會一直開關，貓咪就會有更多逃跑的機會，即使有裝紗窗，貓咪的重量也可能導致紗窗脫落，如果住在高樓層，貓咪就可能墜樓喪命。所以如果想將貓跳台放在窗戶附近，請務必做好窗戶防護。
4.床邊或嬰兒床附近：放置貓跳台時，不僅要確保貓咪的安全，也要確保與貓一起生活的人類的安全，因為貓跳台使用久了也有倒塌或斷裂的風險，如果放在床邊或嬰兒床附近，就可能會弄傷睡覺的主人跟寶寶。
""",
"""
台南市成功大學勝利校區體育館今天晚上9點多發生火災，消防局接獲報案派遣13車27人前往搶救，現場為2樓健身房的天花板管線冒煙，人員都已進行疏散，消防人員出水滅火，火勢約半小時後撲滅。
消防人員說，可能是體育館空調設備故障引起火災；台南市消防局說，起火真正原因需由消災調查科人員進一步鑑定查明。
台南市消防局救災救護指揮中心今晚9點32分接獲報案，成功大學勝利校區體育館發生火災，中心派遣東門等消防分隊13車27人前往搶救，消防人員到達現場回報為2樓健身房的天花板管線冒煙，已進行疏散。義消到達外觀有濃煙，校方有初期滅火。
據了解，2樓為健身房使用，無延燒之虞，館內人員皆撤出。消防人員出水降溫，及排煙。火勢於晚上10點3分撲滅。
""",
"""
超衰！50歲陳姓男子在高雄鳳山租屋，未料某天睡著時，床頭夾式檯燈電源線短路起火，火勢一發不可收拾，一路延燒到至少6處樓層，雖未釀成傷亡，但經消防火調科調查，證據都指向陳男釀災可能性最大，被5名租客聯合提告，法院審理後，將他依法判4個月有期徒刑，得易科罰金。
判決指出，50歲陳男高雄鳳山租屋，2023年初凌晨睡覺時，臥室床頭夾式檯燈電源線短路起火，瞬間燃燒周邊木質家具及電器，火勢一發不可收拾，一路延燒到至少6處樓層，造成建物客廳、廚房、廁所、床墊、家具等物被燒毀。
消防撲滅火勢後，雖未釀成人員傷亡，但受害5名房客聯合對陳男提告；經火調科調查，火災起火點位在陳男住處臥室靠客廳走廊床頭櫃附近，起火原因以電氣因素引燃可能性最大，他因此吃上官司。
陳男警詢及偵訊時，都坦承此事，被檢方依失火燒燬他人之物等罪嫌起訴，他也在審理過程中自白。
法官認為，陳男使用電器用品時，未注意夾式檯燈電源線使用狀況，導致火災發生，除燒燬自己租用住宅外，還波及隔鄰其他住戶，造成多人損失，考量坦承犯行，態度尚可，審理後，將他依犯失火燒燬現供人使用之住宅罪，處4個月有期徒刑，得易科罰金。
""",
"""
桃園楊梅幼獅工業區一間工廠深夜發生大火，濃煙竄天，消防出動66人漏夜搶救，派出高效能化學消防車「美洲豹」馳援搶救，幸好沒有人員受傷，詳細起火原因，有待火調科後續釐清。
鐵皮工廠深夜時分冒出熊熊烈火，濃煙不斷竄出火光照亮天空，燃燒面積相當大，附近住戶飽受驚嚇，甚至有住戶在火災發生後，聽工廠內傳出猛烈爆炸聲。
5號晚上11點桃園楊梅獅一路，一間鐵皮工廠發生火警，警消獲報後派遣28輛消防車、66人到場，立刻佈水線搶救，同時派出高效能化學車「美洲豹」到場馳援。這間工廠位於幼獅工業區內，是塑膠產業公司，面積將近千坪，火勢蔓延非常快速，詳細起火原因，有待火調科後續釐清。
"""]

In [142]:
class boolProcessor:
    def __call__(self, inputs):
        # inputs 是一個 AIMessage 物件，獲取其內容
        response = inputs.content
        # 找到 <答案>: 之後的內容
        question_end = response.find('請判斷提供的新聞內容是否與火災有關')
        # answer_start = response.find("<答案>:") + len("<答案>:")
        # text = response[answer_start:].replace('\n', '').strip()
        question = response[:question_end]
        text = response.split('[/INST]')[1].replace('\n', '').strip()
        try:
            # 構建结果字典
            response_json = {
                "text": test,
                "question": question
                }
        except Exception as e:
            # 如果轉換失敗，返回原始回應
            response_json = {"error": str(e), "response": response}
        return response_json

In [153]:
class JsonProcessor:
    def __call__(self, inputs):
        # inputs 是一個 AIMessage 物件，獲取其內容
        response = inputs.content
        # 找到 <答案>: 之後的內容
        answer_start = response.find("<答案>:") + len("<答案>:")
        text = response[answer_start:].replace('\n', '').strip()
        if_fire = re.search(r"與火災是否相關[:：]([\w\s]+)", text) # [\w\s]+ 是一個字類符號
        if not if_fire=='是' :
          return "此問題無法回答"
        try:
            # 使用正則表達式提取信息
            fire_location = re.search(r"起火點[:：]([\w\s]+)", text) # [\w\s]+ 是一個字類符號
            fire_cause = re.search(r"起火原因[:：]([\w\s]+)", text)
            fire_time = re.search(r"起火時間[:：]([\w\s]+)", text)
            death_info = re.search(r"是否有人死亡[:：](\w+)", text)

            # 構建结果字典
            response_json = {
                "起火點": fire_location.group(1).strip() if fire_location else "未知",
                "起火原因": fire_cause.group(1).strip() if fire_cause else "未知",
                "起火時間": fire_time.group(1).strip() if fire_time else "未知",
                "是否有人死亡": False if death_info and death_info.group(1).strip().lower() in ["false", "否"] else True
            }
        except Exception as e:
            # 如果轉換失敗，返回原始回應
            response_json = {"error": str(e), "response": response}
        return response_json

In [154]:
template = """新聞內容: {question}
        請判斷提供的新聞內容是否與火災有關，回達"是"或"不是"
        <答案>: """
prompt = PromptTemplate(template=template, input_variables=["question"])

template2 = """
        新聞內容: {question}
        請將提供的火災新聞描述中萃取出
        起火點
        起火原因
        起火時間
        是否有人死亡(bool)
        與火災是否相關{text}
        並以JSON 格式回傳，是一個有4個中文key值的dictionary
        <答案>: """
prompt2 = PromptTemplate(template=template, input_variables=["text", "question"])



In [155]:
llm_chain = prompt | chat_llm | boolProcessor() | prompt2 | chat_llm | JsonProcessor()

In [156]:
llm_chain.invoke({"question": news[0]})

'此問題無法回答'

In [157]:
llm_chain.invoke({"question": news[1]})

'此問題無法回答'

In [ ]:
llm_chain.invoke({"question": news[0]})